<a href="https://colab.research.google.com/github/KishenPatel97/Book_Summarization/blob/main/bleu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import nltk
from nltk.translate import bleu
import pandas as pd
from nltk.translate.bleu_score import SmoothingFunction
nltk.download('punkt')
from nltk import word_tokenize
# from summarize import processBook

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [60]:
hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to help students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and vocabulary at the bottom of the page, comment for a chance to be mentioned on cnn student news. you must be a teacher or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests students' knowledge of even ts in the news"

hyp1 = "This text is just for testing purposes"

ref1 = "The article is just meant for evaluation"
# Requires text in a tokenized format
score1 = bleu([word_tokenize(reference)], word_tokenize(hypothesis))
print(score1)

chencherry = SmoothingFunction()
score2 = bleu([word_tokenize(ref1)], word_tokenize(hyp1), smoothing_function = chencherry.method1)
print(score2)

0.17607804141687589
0.07730551756939454


In [61]:
def processBook(fname):
    """ Function to process .txt books, esp. from Gutenberg.
    PARAMS: fname (str) - filepath to book to be processed into a string
    RETURNS: (str) - the string object containing the text
    """
    book = open(fname)      # open book file
    book_lines = []         # initialize storage
    for line in book.readlines():       # iterate through each line
        book_lines.append(line.strip())     # process each line

    return " ".join(book_lines)         # return processed lines

In [62]:
def evaluateSummary(controltext1, controltext2, summarytext, data_score):
    """Helper function to run summarization experiments
    PARAMS: controltext1 (str) - path to Cliff's Notes summary
            controltext2 (str) - path to Grade Saver summary
            summarytext (str) - path to generated summary to evaluate
    """
    control1 = processBook(controltext1)
    control2 = processBook(controltext2)
    summary = processBook(summarytext)

    chencherry = SmoothingFunction()
    # Requires text in a tokenized format
    # Calculate the upper bound (Bleu of 2 manually created summaries)
    ub_scores = bleu([word_tokenize(control1)], word_tokenize(control2), weights = (1, 0, 0, 0), smoothing_function = chencherry.method1)
    ub_scores_bi = bleu([word_tokenize(control1)], word_tokenize(control2), weights = (0, 1, 0, 0), smoothing_function = chencherry.method1)
    ub_scores_cumal = bleu([word_tokenize(control1)], word_tokenize(control2), smoothing_function = chencherry.method1)

    ub_scores2 = bleu([word_tokenize(control2)], word_tokenize(control1), weights = (1, 0, 0, 0), smoothing_function = chencherry.method1)
    ub_scores2_bi = bleu([word_tokenize(control2)], word_tokenize(control1), weights = (0, 1, 0, 0), smoothing_function = chencherry.method1)
    ub_scores2_cumal = bleu([word_tokenize(control2)], word_tokenize(control1), smoothing_function = chencherry.method1)

    sum_scores_cn = bleu([word_tokenize(summary)], word_tokenize(control1), weights = (1, 0, 0, 0), smoothing_function = chencherry.method1)
    sum_scores_cn_bi = bleu([word_tokenize(summary)], word_tokenize(control1), weights = (0, 1, 0, 0), smoothing_function = chencherry.method1)
    sum_scores_cn_cumal = bleu([word_tokenize(summary)], word_tokenize(control1), smoothing_function = chencherry.method1)

    sum_scores_gs = bleu([word_tokenize(summary)], word_tokenize(control2), weights = (1, 0, 0, 0), smoothing_function = chencherry.method1)
    sum_scores_gs_bi = bleu([word_tokenize(summary)], word_tokenize(control2), weights = (0, 1, 0, 0), smoothing_function = chencherry.method1)
    sum_scores_gs_cumal = bleu([word_tokenize(summary)], word_tokenize(control2), smoothing_function = chencherry.method1)

    data_score.append([controltext1, ub_scores,ub_scores_bi,ub_scores_cumal,ub_scores2,ub_scores2_bi,ub_scores2_cumal,sum_scores_cn,sum_scores_cn_bi,sum_scores_cn_cumal,sum_scores_gs,sum_scores_gs_bi,sum_scores_gs_cumal])

    #print("Scores for {}".format(controltext1[18:]))
    print(f"Scores for {controltext1}")
    print("Upper Bound:")
    print(f'unigram:    {ub_scores}')
    print(f'bigram:     {ub_scores_bi}')
    print(f'cumalitive  {ub_scores_cumal}')
    print(f'\nunigram:    {ub_scores2}')
    print(f'bigram:     {ub_scores2_bi}')
    print(f'cumalitive  {ub_scores2_cumal}')
    print("\nCliff's Notes:")
    print(f'unigram:    {sum_scores_cn}')
    print(f'bigram:     {sum_scores_cn_bi}')
    print(f'cumalitive: {sum_scores_cn_cumal}')
    print("\nGradeSaver:")
    print(f'unigram:    {sum_scores_gs}')
    print(f'bigram:     {sum_scores_gs_bi}')
    print(f'cumalitive: {sum_scores_gs_cumal}')

In [63]:
# storing scores
data_scores = []

In [64]:
evaluateSummary("cn_alice.txt",
                "gs_alice.txt",
                "alice_clean_summary_5_sent.txt", data_scores)

Scores for cn_alice.txt
Upper Bound:
unigram:    0.08303134720918029
bigram:     0.032459204484857264
cumalitive  0.01419038319425634

unigram:    0.2351590685470646
bigram:     0.09186351706036745
cumalitive  0.04014594269451768

Cliff's Notes:
unigram:    0.22531977697605776
bigram:     0.043963254593175856
cumalitive: 0.009899934612300541

GradeSaver:
unigram:    0.3113996147466913
bigram:     0.0633306237998558
cumalitive: 0.018890679970605964


In [65]:
evaluateSummary("cn_david.txt",
                "gs_david.txt",
                "david_summary_5_sent.txt", data_scores)

Scores for cn_david.txt
Upper Bound:
unigram:    0.3392307692307692
bigram:     0.13086989992301773
cumalitive  0.056846519709643065

unigram:    0.22932578989714053
bigram:     0.08854956898635996
cumalitive  0.038480971039978065

Cliff's Notes:
unigram:    0.0003384215398488578
bigram:     7.61765338760912e-05
cumalitive: 9.16240213639191e-06

GradeSaver:
unigram:    0.030824760269202063
bigram:     0.007054661419728189
cumalitive: 0.0007850360748045481


In [66]:
evaluateSummary("cn_expectations.txt",
                "gs_expectations.txt",
                "expectations_summary_5_sent.txt", data_scores)

Scores for cn_expectations.txt
Upper Bound:
unigram:    0.46613545816733065
bigram:     0.18768115942028984
cumalitive  0.08229812443984479

unigram:    0.40850835159465515
bigram:     0.16451470490322925
cumalitive  0.07214756792939755

Cliff's Notes:
unigram:    0.08922774628490296
bigram:     0.017559975960960143
cumalitive: 0.004375727845936414

GradeSaver:
unigram:    0.24825586487864265
bigram:     0.05509079646941433
cumalitive: 0.005851759233539406


In [67]:
evaluateSummary("cn_frank.txt",
                "gs_frank.txt",
                "frank_summary_5_sent.txt", data_scores)

Scores for cn_frank.txt
Upper Bound:
unigram:    0.2972972972972973
bigram:     0.10815682739972962
cumalitive  0.041622924169551496

unigram:    0.17754126774891524
bigram:     0.06462988085014275
cumalitive  0.02487986089601523

Cliff's Notes:
unigram:    0.05603954269824744
bigram:     0.012466622282453613
cumalitive: 0.00155635219558151

GradeSaver:
unigram:    0.36239576081712777
bigram:     0.08307358719559678
cumalitive: 0.019663668252279786


In [68]:
evaluateSummary("cn_huckfin.txt",
                "gs_huckfin.txt",
                "huckfin_summary_5_sent.txt", data_scores)

Scores for cn_huckfin.txt
Upper Bound:
unigram:    0.2803921568627451
bigram:     0.11378126532614026
cumalitive  0.05191206501684533

unigram:    0.1484987615856758
bigram:     0.06030670128575214
cumalitive  0.027525336504052856

Cliff's Notes:
unigram:    0.007189965489960233
bigram:     0.001587072072113309
cumalitive: 0.0005413107081658332

GradeSaver:
unigram:    0.1900743403645806
bigram:     0.03954812083434094
cumalitive: 0.010358564672349246


In [69]:
evaluateSummary("cn_pride.txt",
                "gs_pride.txt",
                "pride_summary_5_sent.txt", data_scores)

Scores for cn_pride.txt
Upper Bound:
unigram:    0.3231707317073171
bigram:     0.14449104079298514
cumalitive  0.06221505030388924

unigram:    0.18140364266272244
bigram:     0.08115244351259103
cumalitive  0.03495262654841914

Cliff's Notes:
unigram:    0.0030411666284463415
bigram:     0.0009190445048768735
cumalitive: 0.0002891016187865049

GradeSaver:
unigram:    0.12524494640551825
bigram:     0.03017351447818505
cumalitive: 0.009384811020106847


In [70]:
evaluateSummary("cn_scarlet.txt",
                "gs_scarlet.txt",
                "scarlet_summary_5_sent.txt", data_scores)

Scores for cn_scarlet.txt
Upper Bound:
unigram:    0.4584786624933773
bigram:     0.1311605624799302
cumalitive  0.048563373167915966

unigram:    0.47834456207892206
bigram:     0.13680154142581888
cumalitive  0.050644147959759196

Cliff's Notes:
unigram:    0.0875012301211657
bigram:     0.020947665282911383
cumalitive: 0.002724053311001991

GradeSaver:
unigram:    0.036919125329569334
bigram:     0.0077314711062871
cumalitive: 0.001222035977226448


In [71]:
evaluateSummary("cn_tale.txt",
                "gs_tale.txt",
                "tale_summary_5_sent.txt", data_scores)

Scores for cn_tale.txt
Upper Bound:
unigram:    0.39036978012722634
bigram:     0.1327118069258435
cumalitive  0.06417368020060649

unigram:    0.43161634103019536
bigram:     0.14666666666666667
cumalitive  0.07090525812334236

Cliff's Notes:
unigram:    0.030856542133496704
bigram:     0.006461876835767729
cumalitive: 0.0007751729475069887

GradeSaver:
unigram:    0.004318944609210348
bigram:     0.001029258237433524
cumalitive: 0.00013357862982062282


In [72]:
evaluateSummary("cn_treasure.txt",
                "gs_treasure.txt",
                "treasure_summary_5_sent.txt", data_scores)

Scores for cn_treasure.txt
Upper Bound:
unigram:    0.49544922544410486
bigram:     0.1454979131243865
cumalitive  0.0615375452620419

unigram:    0.4977738201246661
bigram:     0.14616755793226383
cumalitive  0.06181800524638476

Cliff's Notes:
unigram:    0.07013434908391808
bigram:     0.017628862830753707
cumalitive: 0.005381770884365949

GradeSaver:
unigram:    0.05029364404223632
bigram:     0.014273807990794614
cumalitive: 0.009179688951930455


In [73]:
evaluateSummary("cn_warpeace.txt",
                "gs_warpeace.txt",
                "warpeace_summary_5_sent.txt", data_scores)

Scores for cn_warpeace.txt
Upper Bound:
unigram:    0.3752913752913753
bigram:     0.10447761194029852
cumalitive  0.030991563369445897

unigram:    0.3365555744093594
bigram:     0.09371762107043731
cumalitive  0.027803312488139758

Cliff's Notes:
unigram:    5.596751883779981e-13
bigram:     2.1493342197624413e-13
cumalitive: 5.84764317997896e-14

GradeSaver:
unigram:    1.4599211339103648e-08
bigram:     5.6900466883542325e-09
cumalitive: 1.8305523297427825e-09


In [74]:
# saving scores
da = pd.DataFrame(data_scores)
da.to_csv('scores5_sent.csv', index = False)
da

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,cn_alice.txt,0.083031,0.032459,0.014190,0.235159,0.091864,0.040146,2.253198e-01,4.396325e-02,9.899935e-03,3.113996e-01,6.333062e-02,1.889068e-02
1,cn_david.txt,0.339231,0.130870,0.056847,0.229326,0.088550,0.038481,3.384215e-04,7.617653e-05,9.162402e-06,3.082476e-02,7.054661e-03,7.850361e-04
2,cn_expectations.txt,0.466135,0.187681,0.082298,0.408508,0.164515,0.072148,8.922775e-02,1.755998e-02,4.375728e-03,2.482559e-01,5.509080e-02,5.851759e-03
3,cn_frank.txt,0.297297,0.108157,0.041623,0.177541,0.064630,0.024880,5.603954e-02,1.246662e-02,1.556352e-03,3.623958e-01,8.307359e-02,1.966367e-02
4,cn_huckfin.txt,0.280392,0.113781,0.051912,0.148499,0.060307,0.027525,7.189965e-03,1.587072e-03,5.413107e-04,1.900743e-01,3.954812e-02,1.035856e-02
5,cn_pride.txt,0.323171,0.144491,0.062215,0.181404,0.081152,0.034953,3.041167e-03,9.190445e-04,2.891016e-04,1.252449e-01,3.017351e-02,9.384811e-03
6,cn_scarlet.txt,0.458479,0.131161,0.048563,0.478345,0.136802,0.050644,8.750123e-02,2.094767e-02,2.724053e-03,3.691913e-02,7.731471e-03,1.222036e-03
7,cn_tale.txt,0.390370,0.132712,0.064174,0.431616,0.146667,0.070905,3.085654e-02,6.461877e-03,7.751729e-04,4.318945e-03,1.029258e-03,1.335786e-04
8,cn_treasure.txt,0.495449,0.145498,0.061538,0.497774,0.146168,0.061818,7.013435e-02,1.762886e-02,5.381771e-03,5.029364e-02,1.427381e-02,9.179689e-03
9,cn_warpeace.txt,0.375291,0.104478,0.030992,0.336556,0.093718,0.027803,5.596752e-13,2.149334e-13,5.847643e-14,1.459921e-08,5.690047e-09,1.830552e-09
